In [1]:
import jax.numpy as jnp
import jax.random as jrandom
import gymnasium as gym
from stable_baselines3 import PPO
from utils import step_estimation,current_state_and_process,process_covariance,kalman_function

In [2]:
env=gym.make("CartPole-v1")
env._max_episode_steps=1000

In [57]:
model = PPO("MlpPolicy",
            env,
            verbose=1)

model.learn(total_timesteps=100000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 22.2     |
|    ep_rew_mean     | 22.2     |
| time/              |          |
|    fps             | 1558     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 24.9         |
|    ep_rew_mean          | 24.9         |
| time/                   |              |
|    fps                  | 963          |
|    iterations           | 2            |
|    time_elapsed         | 4            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0071497834 |
|    clip_fraction        | 0.087        |
|    clip_range           | 0.2          |
|    en

In [58]:
model.save("ppo_cartpole_100k")

In [4]:
model=PPO.load("ppo_cartpole_100k")
rewardss = 0
episodes = 10
for ep in range(episodes):
    rewards = 0
    state=env.reset()[0]
    while True:
        #env.render()
        action = model.predict(state)[0]
        state, reward, done,truncuated, info = env.step(action)
        rewards += reward
        if done or truncuated:
            break
    print(f"Episode: {ep+1}, Rewards: {rewards}")
    rewardss += rewards
print(f"mean_rewards: {rewardss/(episodes)}")

Episode: 1, Rewards: 170.0
Episode: 2, Rewards: 144.0
Episode: 3, Rewards: 210.0
Episode: 4, Rewards: 184.0
Episode: 5, Rewards: 223.0
Episode: 6, Rewards: 167.0
Episode: 7, Rewards: 142.0
Episode: 8, Rewards: 171.0
Episode: 9, Rewards: 159.0
Episode: 10, Rewards: 158.0
mean_rewards: 172.8


In [10]:
delta_t = .1
m=.1
M=1
g=9.8
l=.5
A=jnp.array([[0,1,0,0],
             [0,0,m*g*delta_t/M,0],
             [0,0,0,1],
             [0,0,(m+M)*g*delta_t/(M*l),0]])

B=jnp.array([[0],[delta_t/M],[0],[delta_t/(M*l)]])

H=jnp.array([[0,1,0,0],
             [0,0,0,1]])

R=jnp.eye(H.shape[0])*0.001
C=jnp.eye(H.shape[0])
Q=jnp.eye(A.shape[0])*0.001
#Q=jnp.zeros(A.shape)
P0=jnp.ones(A.shape[0])
mean=0
std_dev=0.005
key=jrandom.PRNGKey(42)
Z=jrandom.normal(key, shape=(H.shape[0],1)) * std_dev + mean
#Z=jnp.zeros((H.shape[0],1))
w_k = jnp.zeros((A.shape[0],1))

In [17]:
jnp.expand_dims(env.reset()[0], axis=-1).shape

(4, 1)

In [18]:
model=PPO.load("ppo_cartpole_100k")
P=P0
rewardss = 0
episodes = 10
for ep in range(episodes):
    rewards = 0
    state_p=jnp.expand_dims(env.reset()[0], axis=-1)
    while True:
        #env.render()
        action,_ = model.predict(state_p.reshape(1,4).squeeze())
        action2= -1 if action == 0 else 1
        u_k = jnp.array([[action2]])
        state_p=step_estimation(A,B,state_p,u_k,w_k)
        print(state_p.shape)
        P=process_covariance(A,P,Q)
        state_m, reward, done, truncuated, info = env.step(action)
        K=kalman_function(P,H,R)
        state_p,P=current_state_and_process(K,H,C,H @ state_m.reshape(4,1),state_p,P,Z)
        #print(state_m.reshape((4,1)) - state_p)
        rewards += reward
        if done or truncuated:
            break
    print(f"Episode: {ep+1}, Rewards: {rewards}")
    rewardss += rewards
print(f"mean_rewards: {round(rewardss/(episodes),2)}")

(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)